In [3]:
library(ggplot2)
library(stats)
library(dplyr)
library(tidyverse)
library(RColorBrewer)
library(paletteer)
library(ComplexHeatmap)
library(circlize)
library(grid)

# Reading input

In [24]:
mat<-read.csv("06_Arc_Plot_Data_Prepprocessing/mat.csv",row.names=1,check.names = FALSE)
row_annot<-read.csv("06_Arc_Plot_Data_Prepprocessing//row_anno.csv",row.names=1,check.names = FALSE)

In [25]:
df<-cbind(row_annot,mat)
colnames(df)[12]<-"Plasma Cell Freq Changes"
colnames(df)[1:3]<-c("subjectGuid","Flu_Year","Flu_Day")
colnames(df)[7:10]<-paste0("Response_Group-",colnames(df)[7:10])
colnames(df)[13:16]<-paste0("Response_Score-",colnames(df)[13:16])
write.csv(df ,"/home/workspace/Source_Data/Fig4b.csv", row.names=FALSE)

In [40]:
pdf('FigureB.pdf', width =200/25.4, height = 120/25.4)

circos.clear()
circos.par(track.height = 0.1, start.degree = 180, gap.after = c(3.5, 180))
col_fun1 <- colorRamp2(seq(median(mat[, 1], na.rm = TRUE) - 2 * mad(mat[, 1], na.rm = TRUE), median(mat[, 1], na.rm = TRUE) + 2 * mad(mat[, 1], na.rm = TRUE),
    length.out = 11
), rev(brewer.pal(11, "Spectral")))
col_fun2 <- colorRamp2(seq(median(mat[, 2], na.rm = TRUE) - 2 * mad(mat[, 2], na.rm = TRUE), median(mat[, 2], na.rm = TRUE) + 2 * mad(mat[, 2], na.rm = TRUE),
    length.out = 11
), rev(brewer.pal(11, "Spectral")))
col_fun3 <- colorRamp2(seq(median(mat[, 3], na.rm = TRUE) - 2 * mad(mat[, 3], na.rm = TRUE), median(mat[, 3], na.rm = TRUE) + 2 * mad(mat[, 3], na.rm = TRUE),
    length.out = 11
), rev(brewer.pal(11, "Spectral")))
col_fun4 <- colorRamp2(seq(median(mat[, 4], na.rm = TRUE) - 2 * mad(mat[, 4], na.rm = TRUE), median(mat[, 4], na.rm = TRUE) + 2 * mad(mat[, 4], na.rm = TRUE),
    length.out = 11
), rev(brewer.pal(11, "Spectral")))


set_track_gap(cm_h(0.1))

annotation_levels <- unique(row_annot$V2)
if (length(annotation_levels) < 3) {
    annotation_colors <- setNames(brewer.pal(3, "Set2")[1:length(annotation_levels)], annotation_levels)
} else {
    annotation_colors <- setNames(brewer.pal(length(annotation_levels), "Set2"), annotation_levels)
}
anno_mat <- as.matrix(as.numeric(factor(row_annot$V2, levels = annotation_levels)))
col_fun_anno <- colorRamp2(seq(1, length(annotation_levels)), annotation_colors)
circos.heatmap(anno_mat, col = col_fun_anno, cluster = FALSE, split = row_annot$V2, track.height = 0.01)


annotation_levels <- unique(row_annot$cohort.cohortGuid)
annotation_colors <- c(BR1 = "#35978f", BR2 = "#bf812d")[levels(factor(row_annot$cohort.cohortGuid))]
anno_mat <- as.matrix(as.numeric(factor(row_annot$cohort.cohortGuid)))
col_fun_anno <- colorRamp2(seq(1, length(annotation_levels)), annotation_colors)
circos.heatmap(anno_mat, col = col_fun_anno, cluster = FALSE, cell.border = "white", cell.lwd = 0.1,  track.height = 0.02)

annotation_levels <- unique(row_annot$subject.biologicalSex)
annotation_colors <- c(Male = "#5aae61", Female = "#9970ab")[levels(factor(row_annot$subject.biologicalSex))]
anno_mat <- as.matrix(as.numeric(factor(row_annot$subject.biologicalSex)))
col_fun_anno <- colorRamp2(seq(1, length(annotation_levels)), annotation_colors)
circos.heatmap(anno_mat, col = col_fun_anno, cluster = FALSE, cell.border = "white", cell.lwd = 0.2, track.height = 0.02)

annotation_levels <- unique(row_annot$CMV)
annotation_colors <- c(Positive = "#d6604d", Negative = "#4393c3")[levels(factor(row_annot$CMV))]
anno_mat <- as.matrix(as.numeric(factor(row_annot$CMV)))
col_fun_anno <- colorRamp2(seq(1, length(annotation_levels)), annotation_colors)
circos.heatmap(anno_mat, col = col_fun_anno, cluster = FALSE, cell.border = "white", cell.lwd = 0.2,  track.height = 0.02)


circos.trackPlotRegion(
  factors = row_annot$V2,
  ylim = c(0, max(row_annot$`Plasma cell`, na.rm = TRUE)), 
  track.height = 0.06,
  panel.fun = function(x, y) {
    sector.name <- get.cell.meta.data("sector.index")
    
    idx <- row_annot$V2 == sector.name
    
    x_val <- seq_len(sum(idx)) - 0.5
    
    y_val <- row_annot$`Plasma cell`[idx]
    
    # now draw bars in this sector
    circos.barplot(
      y_val, 
      x_val, 
      col = 'black', 
      border = 'black'
    )
  }
)

set_track_gap(cm_h(0.3))

annotation_colors <- c(`low responder` = "#bdd7e7",`middle responder` = "#6baed6",`high responder` = "#08519c")

annotation_levels <- intersect(unique(row_annot$`Flu B/Phuket HA`), names(annotation_colors))
row_annot$`Flu B/Phuket HA` <- factor(row_annot$`Flu B/Phuket HA`, levels = annotation_levels)
anno_mat <- matrix(as.numeric(row_annot$`Flu B/Phuket HA`))
col_fun_anno <- colorRamp2(seq_along(annotation_levels), annotation_colors[annotation_levels])

circos.heatmap(anno_mat, col = col_fun_anno, cluster = FALSE, cell.border = "white", cell.lwd = 0.8, track.height = 0.02)
set_track_gap(cm_h(0.1))

circos.heatmap(mat[, 1], col = col_fun1, cluster = FALSE, split = row_annot$V2, track.height = 0.06)
set_track_gap(cm_h(0.2))

annotation_levels <- intersect(unique(row_annot$`Flu B/Washington HA`), names(annotation_colors))
row_annot$`Flu B/Washington HA` <- factor(row_annot$`Flu B/Washington HA`, levels = annotation_levels)
anno_mat <- matrix(as.numeric(row_annot$`Flu B/Washington HA`))
col_fun_anno <- colorRamp2(seq_along(annotation_levels), annotation_colors[annotation_levels])

circos.heatmap(anno_mat, col = col_fun_anno, cluster = FALSE, cell.border = "white", cell.lwd = 0.8,  track.height = 0.02)
set_track_gap(cm_h(0.1))

circos.heatmap(mat[, 2], col = col_fun2, cluster = FALSE, split = row_annot$V2, track.height = 0.06)
set_track_gap(cm_h(0.2))

annotation_levels <- intersect(unique(row_annot$`A/Guangdong`), names(annotation_colors))
row_annot$`A/Guangdong` <- factor(row_annot$`A/Guangdong`, levels = annotation_levels)
anno_mat <- matrix(as.numeric(row_annot$`A/Guangdong`))
col_fun_anno <- colorRamp2(seq_along(annotation_levels), annotation_colors[annotation_levels])

circos.heatmap(anno_mat, col = col_fun_anno, cluster = FALSE, cell.border = "white", cell.lwd = 0.8,  track.height = 0.02)
set_track_gap(cm_h(0.1))

circos.heatmap(mat[, 3], col = col_fun3, cluster = FALSE, split = row_annot$V2, track.height = 0.06)
set_track_gap(cm_h(0.2))


annotation_levels <- intersect(unique(row_annot$`A/Cambodia`), names(annotation_colors))
row_annot$`A/Cambodia` <- factor(row_annot$`A/Cambodia`, levels = annotation_levels)
anno_mat <- matrix(as.numeric(row_annot$`A/Cambodia`))
col_fun_anno <- colorRamp2(seq_along(annotation_levels), annotation_colors[annotation_levels])

circos.heatmap(anno_mat, col = col_fun_anno, cluster = FALSE, cell.border = "white", cell.lwd = 0.8,  track.height = 0.02)
set_track_gap(cm_h(0.1))



circos.heatmap(mat[, 4], col = col_fun4, cluster = FALSE, split = row_annot$V2, track.height = 0.06)
set_track_gap(cm_h(0.2))

circos.clear()

dev.off()


Warning message:
“`convert_length()` only works when aspect of the coordinate is 1.”
Note: 44 points are out of plotting region in sector '2020-2021', track
'5'.

Note: 46 points are out of plotting region in sector '2021-2022', track
'5'.



pdf 
  2

In [41]:
pdf('FigureB_Legend.pdf', width =200/25.4, height = 120/25.4)


annotation_colors_v2 <- if(length(unique(row_annot$V2)) < 3) {
  setNames(brewer.pal(3, "Set2")[1:length(unique(row_annot$V2))], unique(row_annot$V2))
} else {
  setNames(brewer.pal(length(unique(row_annot$V2)), "Set2"), unique(row_annot$V2))
}
annotation_colors_cohort <- c(BR1 = "#35978f", BR2 = "#bf812d")[levels(factor(row_annot$cohort.cohortGuid))]
annotation_colors_sex <- c(Male = "#5aae61", Female = "#9970ab")[levels(factor(row_annot$subject.biologicalSex))]
annotation_colors_CMV <- c(Positive = "#d6604d", Negative = "#4393c3")[levels(factor(row_annot$CMV))]
annotation_colors_fluPhuket <- annotation_colors[intersect(unique(row_annot$`Flu B/Phuket HA`), names(annotation_colors))]
annotation_colors_fluWash <- annotation_colors[intersect(unique(row_annot$`Flu B/Washington HA`), names(annotation_colors))]
annotation_colors_guangdong <- annotation_colors[intersect(unique(row_annot$`A/Guangdong`), names(annotation_colors))]
annotation_colors_cambodia <- annotation_colors[intersect(unique(row_annot$`A/Cambodia`), names(annotation_colors))]

lgd1 <- Legend(title = "mat[,1]", col_fun = col_fun1)
lgd2 <- Legend(title = "mat[,2]", col_fun = col_fun2)
lgd3 <- Legend(title = "mat[,3]", col_fun = col_fun3)
lgd4 <- Legend(title = "mat[,4]", col_fun = col_fun4)
lgd_v2 <- Legend(title = "V2", at = names(annotation_colors_v2), legend_gp = gpar(fill = annotation_colors_v2))
lgd_cohort <- Legend(title = "cohort", at = names(annotation_colors_cohort), legend_gp = gpar(fill = annotation_colors_cohort))
lgd_sex <- Legend(title = "sex", at = names(annotation_colors_sex), legend_gp = gpar(fill = annotation_colors_sex))
lgd_CMV <- Legend(title = "CMV", at = names(annotation_colors_CMV), legend_gp = gpar(fill = annotation_colors_CMV))
lgd_fluPhuket <- Legend(title = "Flu B/Phuket HA", at = names(annotation_colors_fluPhuket), legend_gp = gpar(fill = annotation_colors_fluPhuket))
lgd_fluWash <- Legend(title = "Flu B/Washington HA", at = names(annotation_colors_fluWash), legend_gp = gpar(fill = annotation_colors_fluWash))
lgd_guangdong <- Legend(title = "A/Guangdong", at = names(annotation_colors_guangdong), legend_gp = gpar(fill = annotation_colors_guangdong))
lgd_cambodia <- Legend(title = "A/Cambodia", at = names(annotation_colors_cambodia), legend_gp = gpar(fill = annotation_colors_cambodia))

all_lgd <- packLegend(lgd1, lgd2, lgd3, lgd4, lgd_v2, lgd_cohort, lgd_sex, lgd_CMV,
                      lgd_fluPhuket, lgd_fluWash, lgd_guangdong, lgd_cambodia,
                      direction = "horizontal")
draw(all_lgd, x = unit(0.5, "npc"), y = unit(0.5, "npc"), just = "center")

dev.off()


pdf 
  2